In [0]:
# Google Colabでkaggle.jsonをアップロード
from google.colab import files
files.upload()

In [0]:
# Google Colabでkaggle.jsonをフォルダ移動
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [0]:
# google colabでkaggleのアップデートする
!pip uninstall kaggle
!pip install --upgrade pip
!pip install kaggle==1.5.6
!kaggle -v

In [0]:
# Google Colabでデータセットをダウンロード
!kaggle competitions download -c flower-classification-with-tpus

In [0]:
# ファイルを解凍
!unzip '*.zip'

In [2]:
!pip uninstall keras tensorflow
!pip install tensorflow keras

Uninstalling Keras-2.2.5:
  Would remove:
    /usr/local/lib/python3.6/dist-packages/Keras-2.2.5.dist-info/*
    /usr/local/lib/python3.6/dist-packages/docs/*
    /usr/local/lib/python3.6/dist-packages/keras/*
  Would not remove (might be manually added):
    /usr/local/lib/python3.6/dist-packages/docs/md_autogen.py
    /usr/local/lib/python3.6/dist-packages/docs/update_docs.py
Proceed (y/n)? y
  Successfully uninstalled Keras-2.2.5
Uninstalling tensorflow-2.2.0rc1:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.6/dist-packages/tensorflow-2.2.0rc1.dist-info/*
    /usr/local/lib/python3.6/dist-packages/tensorflow/*
Proceed (y/n)? y
y
  Successfully uninstalled tensorflow-2.2.0rc1
     |████████████████████████████████| 421.8MB 39kB/s 
     |█████████████████

In [1]:
import tensorflow as tf
import tensorflow.keras.backend as K
import numpy as np
from tensorflow.contlib.tpu.python.tpu import keras_support
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Activation, AveragePooling2D, Dense, Dropout, Flatten
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.utils import to_categorical
import os

#ignore warnings
import warnings
warnings.filterwarnings('ignore')

ModuleNotFoundError: ignored